In [139]:
import data
import importlib
importlib.reload(data) #to remive in final .py
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 7it [4:44:28, 2438.32s/it]0<?, ?it/s]
LOADING DATA: 7it [00:32,  3.22s/it]                       

In [140]:
import variables
import contraintes
import importlib
importlib.reload(variables) #to remive in final .py
importlib.reload(contraintes) #to remive in final .py

<module 'contraintes' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\contraintes.py'>

In [141]:
def show_solution():
    print(var_dict)
    for machine in var_dict.keys():
        print(machine)
        # for sillon in var_dict[machine].keys():
            # print(var_dict[machine][sillon].X)


In [150]:
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

final_train_task = dataframes["sillons_df"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
final_train_task = var_dict["essai de frein départ"][final_train_task]
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)

m.update()
m.printStats()
m.presolve()
m.printStats()
m.optimize()
m.write("test.lp")
show_solution(m)

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 889.35it/s]
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:104: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  1][taches_df["Type de train"] == type_train]["Type de tache humaine"].iloc[0]


540
Adding occupation constrainst for: WPY_DEP...


c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:150: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  taches_df[taches_df["Chantier"] == chantier].Ordre.max()]["Type de tache humaine"].iloc[0]
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  taches_df[taches_df["Chantier"] == chantier].Ordre.min()]["Type de tache humaine"].iloc[0]


Adding occupation constrainst for: WPY_REC...
Adding occupation constrainst for: WPY_FOR...
<gurobi.Var *Awaiting Model Update*>

Statistics for modelUnnamed:
  Linear constraint matrix    : 540 Constrs, 70666 Vars, 1080 NZs
  Variable types              : 0 Continuous,
70666 Integer (69586 Binary)
  General constraints         : 139172 Constrs
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 1, 1 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 15, 149 ]
Presolve added 133232 rows and 107732 columns
Presolve time: 2.51s

Statistics for modelUnnamed:
  Linear constraint matrix    : 540 Constrs, 70666 Vars, 1080 NZs
  Variable types              : 0 Continuous,
70666 Integer (69586 Binary)
  General constraints         : 139172 Constrs
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 1, 1 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 15, 149 ]
Gurobi Optimizer version 10.0.0 bu

In [145]:
# tests 
importlib.reload(contraintes)
importlib.reload(variables)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 889.35it/s]
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:104: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  1][taches_df["Type de train"] == type_train]["Type de tache humaine"].iloc[0]


540
Adding occupation constrainst for: WPY_DEP...


c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:150: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  taches_df[taches_df["Chantier"] == chantier].Ordre.max()]["Type de tache humaine"].iloc[0]
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  taches_df[taches_df["Chantier"] == chantier].Ordre.min()]["Type de tache humaine"].iloc[0]


Adding occupation constrainst for: WPY_REC...
Adding occupation constrainst for: WPY_FOR...
